<a href="https://colab.research.google.com/github/neelp93/Coursera_Capstone/blob/main/The%20Battle%20of%20Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute:

I​ love coffee. Based on foursquare reviews & raiting, I will attempt to idnetify the best cafe near me.

Describe the data that you will be using to solve the problem or execute your idea: 

I​ will be using reviews and ratings from foursquare of cafe's in Wellington.

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

In [2]:
CLIENT_ID = 'DMF5HG154PWGCMO2OGKDTO4ORATG3XAK3VVPQJYKMO01BRY3' # your Foursquare ID
CLIENT_SECRET = 'DT5K5T01KFVTMARTYSBK5WMNVNLQF2Z1HM01CBI2DYUFOJAO' # your Foursquare Secret
ACCESS_TOKEN = 'RTDL5XKGJKJ2Q0MPI1YOPUFEHJGQLJVRN3YYSPBZK1SWGQCC' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 300
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DMF5HG154PWGCMO2OGKDTO4ORATG3XAK3VVPQJYKMO01BRY3
CLIENT_SECRET:DT5K5T01KFVTMARTYSBK5WMNVNLQF2Z1HM01CBI2DYUFOJAO


In [3]:
address = '-41.286949, 174.772843'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-41.2870156 174.7727544


In [4]:
search_query = 'Coffee'
radius = 1000
print(search_query + ' .... OK!')

Coffee .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=DMF5HG154PWGCMO2OGKDTO4ORATG3XAK3VVPQJYKMO01BRY3&client_secret=DT5K5T01KFVTMARTYSBK5WMNVNLQF2Z1HM01CBI2DYUFOJAO&ll=-41.2870156,174.7727544&oauth_token=RTDL5XKGJKJ2Q0MPI1YOPUFEHJGQLJVRN3YYSPBZK1SWGQCC&v=20180604&query=Coffee&radius=1000&limit=300'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601df3af8a07476e5f4873df'},
 'notifications': [{'item': {'unreadCount': 3}, 'type': 'notificationTray'}],
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'primary': True,
      'shortName': 'Coffee Shop'}],
    'hasPerk': False,
    'id': '50512403e4b0252d04ec71ee',
    'location': {'address': '115 Dixon St',
     'cc': 'NZ',
     'city': 'Te Aro',
     'country': 'New Zealand',
     'distance': 449,
     'formattedAddress': ['115 Dixon St', 'Te Aro 6011', 'New Zealand'],
     'labeledLatLngs': [{'label': 'display',
       'lat': -41.29094788276601,
       'lng': 174.77395000261714}],
     'lat': -41.29094788276601,
     'lng': 174.77395000261714,
     'postalCode': '6011',
     'state': 'Wellington'},
    'name': 'Flight Coffee Hangar',
    'r

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood
0,50512403e4b0252d04ec71ee,Flight Coffee Hangar,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1612575663,False,115 Dixon St,-41.290948,174.773950,"[{'label': 'display', 'lat': -41.2909478827660...",449,6011,NZ,Te Aro,Wellington,New Zealand,"[115 Dixon St, Te Aro 6011, New Zealand]",NaN,NaN
1,527c27f6498ede09f31fb5e6,Red Rabbit Coffee Co. / Leeds Street Bakery,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1612575663,False,14 Leeds St,-41.292916,174.776946,"[{'label': 'display', 'lat': -41.2929160745904...",744,6011,NZ,Wellington,Wellington,New Zealand,"[14 Leeds St (at Eva St), Wellington 6011, New...",at Eva St,NaN
2,4b6a0af1f964a520bdc32be3,Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1612575663,False,16 The Terrace,-41.279223,174.775473,"[{'label': 'display', 'lat': -41.2792228754837...",896,NaN,NZ,Wellington,Wellington,New Zealand,"[16 The Terrace, Wellington, New Zealand]",NaN,NaN
3,4cf450cb88de37049a42842b,Mojo Coffee Central,"[{'id': '4eb1bea83b7b6f98df247e06', 'name': 'F...",v-1612575663,False,"Shed 13, 37 Customhouse Quay",-41.282885,174.778618,"[{'label': 'display', 'lat': -41.282885, 'lng'...",672,6011,NZ,Wellington,Wellington,New Zealand,"[Shed 13, 37 Customhouse Quay, Wellington 6011...",NaN,NaN
4,5181c388498ee2fcc31205f6,Coffee Merci,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1612575663,False,NaN,-41.288445,174.775401,"[{'label': 'display', 'lat': -41.288445, 'lng'...",272,NaN,NZ,NaN,NaN,New Zealand,[New Zealand],NaN,NaN


In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.sort_values('distance',inplace=True)
dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
36,Mojo Gilmer,Coffee Shop,8 Gilmer Tce,-41.285503,174.774713,"[{'label': 'display', 'lat': -41.2855031010361...",234,6011,NZ,Wellington Central,Wellington,New Zealand,"[8 Gilmer Tce, Wellington Central 6011, New Ze...",NaN,NaN,4b071498f964a520cff622e3
4,Coffee Merci,Café,NaN,-41.288445,174.775401,"[{'label': 'display', 'lat': -41.288445, 'lng'...",272,NaN,NZ,NaN,NaN,New Zealand,[New Zealand],NaN,NaN,5181c388498ee2fcc31205f6
29,Starbucks,Coffee Shop,233-237 Lambton Quay,-41.285828,174.775967,"[{'label': 'display', 'lat': -41.2858276113974...",299,6001,NZ,Wellington,Wellington,New Zealand,"[233-237 Lambton Quay, Wellington 6001, New Ze...",NaN,NaN,4b058778f964a5204f9422e3
33,Mojo Old Bank,Coffee Shop,Old Bank Arcade,-41.286104,174.776224,"[{'label': 'display', 'lat': -41.2861035856492...",307,6011,NZ,Wellington,Wellington,New Zealand,"[Old Bank Arcade (233-237 Lambton Quay), Welli...",233-237 Lambton Quay,NaN,4b05c2b3f964a520a5e222e3
10,Karamu Coffee,Coffee Shop,NaN,-41.289752,174.775134,"[{'label': 'display', 'lat': -41.2897521660354...",363,NaN,NZ,NaN,NaN,New Zealand,[New Zealand],NaN,NaN,512fdf9ce4b0a7dd5315c3d3
7,Columbus Coffee,Café,188 Featherston St,-41.284791,174.776124,"[{'label': 'display', 'lat': -41.2847909776011...",375,6011,NZ,Wellington,Wellington,New Zealand,"[188 Featherston St (at Grey St), Wellington 6...",at Grey St,NaN,52ed7f5811d223d2da8802b2
34,Lamason,Coffee Shop,28 Lombard St,-41.289843,174.776001,"[{'label': 'display', 'lat': -41.2898428935586...",415,6011,NZ,Wellington Central,Wellington,New Zealand,"[28 Lombard St, Wellington Central 6011, New Z...",NaN,NaN,4dc8716bd4c0bf684191524a
12,Francs Coffee,Café,NaN,-41.283768,174.775280,"[{'label': 'display', 'lat': -41.283768, 'lng'...",418,NaN,NZ,NaN,NaN,New Zealand,[New Zealand],NaN,NaN,561ed473498e250c15ed218f
17,Harbour Coffee Lounge,Café,68-86 Jervois Quay,-41.287836,174.777959,"[{'label': 'display', 'lat': -41.2878359840339...",444,NaN,NZ,Wellington,Wellington,New Zealand,"[68-86 Jervois Quay (Telecom House), Wellingto...",Telecom House,NaN,4b0c6de2f964a5200c3d23e3
39,Vicbooks Café,Café,Victoria University,-41.289315,174.768363,"[{'label': 'display', 'lat': -41.2893153805715...",447,6012,NZ,Kelburn,Wellington,New Zealand,"[Victoria University (Kelburn Pde), Kelburn 60...",Kelburn Pde,NaN,4bd9fea62a3a0f47c93aa9b6


In [9]:
dataframe_filtered.name

36                                    Mojo Gilmer
4                                    Coffee Merci
29                                      Starbucks
33                                  Mojo Old Bank
10                                  Karamu Coffee
7                                 Columbus Coffee
34                                        Lamason
12                                  Francs Coffee
17                          Harbour Coffee Lounge
39                                  Vicbooks Café
18                     Margies Tea & Coffee House
0                            Flight Coffee Hangar
6                                  Le Coffee Shop
5                                  Frank's Coffee
15                                  Sierra Coffee
38                                     Cafe Racer
14                                   Crave Coffee
16                                 stories coffee
37                                       Esquires
8                                  Coffee Supreme


In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the Conrad Hotel

# add a red circle marker to represent me
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Me',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Cafes restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [12]:
#The closest cafe to me is Mojo Gilimer
venue_id = '4b071498f964a520cff622e3' # ID of Mojo
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/4b071498f964a520cff622e3?client_id=DMF5HG154PWGCMO2OGKDTO4ORATG3XAK3VVPQJYKMO01BRY3&client_secret=DT5K5T01KFVTMARTYSBK5WMNVNLQF2Z1HM01CBI2DYUFOJAO&oauth_token=RTDL5XKGJKJ2Q0MPI1YOPUFEHJGQLJVRN3YYSPBZK1SWGQCC&v=20180604'

In [13]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'like', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '$', 'priceTier': 1}],
    'name': 'Price',
    'summary': '$',
    'type': 'price'},
   {'count': 3,
    'items': [{'displayName': 'Reservations', 'displayValue': 'No'}],
    'name': 'Reservations',
    'type': 'reservations'},
   {'count': 1,
    'items': [{'displayName': 'Outdoor Seating', 'displayValue': 'No'}],
    'name': 'Outdoor Seating',
    'type': 'outdoorSeating'},
   {'count': 1,
    'items': [{'displayName': 'Wi-Fi', 'displayValue': 'No'}],
    'name': 'Wi-Fi',
    'type': 'wifi'}]},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'bestPhoto': {'createdAt': 1382033212,
  'height': 480,
  'id': '5260273c498ecb186b10abd5',
  'prefix': 'https://fastly.4sqi.net/img/general/',
  'source': {'name': 'Foursquare for BlackBerry',
   'url': 'https://foursquare.com/download/#/blackberry'},
  'suffix': '/7819237

In [14]:
#Getting the rating of Mojo Gilimer
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [15]:
venue_id = '5181c388498ee2fcc31205f6' # ID of Coffee Merci
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [16]:
venue_id = '4b058778f964a5204f9422e3' # ID of Starbucks
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.6


In [17]:
venue_id = '4b05c2b3f964a520a5e222e3' # ID of Mojo at the Old Bank Arcade
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.5


In [18]:
venue_id = '512fdf9ce4b0a7dd5315c3d3' # ID of Karamu Coffee
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Looks like im getting coffee from Starbucks next